In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

Using matplotlib backend: <object object at 0x7f53eb63d340>


In [2]:
import sys
sys.path.append("../../")
import os
from povertymapping.rollout_grids import get_region_filtered_bingtile_grids 
import pandas as pd
import geopandas as gpd

/home/jace/workspace/unicef-ai4d-poverty-mapping/env/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


# Vietnam Model Rollout Part 2 (Generate Roll-out Grids) 

This notebook generates the rollout grid tiles over the country. The output file is used as an input for Step 3, where we will run the trained model over the set of grids.

The generated grids are set at 2.4km (zoom level 14), matching the grids used during model training. The grids are also filtered to only include populated areas based on Meta's High Resolution Settlement Layer (HRSL) data.

## Set country-specific parameters

In [3]:
# Variable REGION must have the correct spelling, spacing and punctuation 
# See table here https://github.com/lukes/ISO-3166-Countries-with-Regional-Codes/blob/master/all/all.csv
REGION = 'viet nam'

ADMIN_LVL = 'ADM2'
ZOOM_LEVEL = 14

## Generate Grids

In [4]:
admin_grids_gdf = get_region_filtered_bingtile_grids(
    REGION, 
    admin_lvl=ADMIN_LVL, 
    quadkey_lvl=ZOOM_LEVEL, 
    use_cache=True,
    n_workers=4
)

2023-03-09 13:04:41.341 | INFO     | povertymapping.rollout_grids:get_region_filtered_bingtile_grids:259 - Loading cached grids file /home/jace/.cache/geowrangler/quadkey_grids/viet nam_14_ADM2_populated_admin_grids.geojson


## Explore Populated Grids

In [5]:
admin_grids_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 50880 entries, 0 to 50879
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   quadkey     50880 non-null  object  
 1   shapeName   50880 non-null  object  
 2   shapeISO    50880 non-null  object  
 3   shapeID     50880 non-null  object  
 4   shapeGroup  50880 non-null  object  
 5   shapeType   50880 non-null  object  
 6   pop_count   50880 non-null  float64 
 7   geometry    50880 non-null  geometry
dtypes: float64(1), geometry(1), object(6)
memory usage: 3.1+ MB


In [6]:
admin_grids_gdf.head(2)

,quadkey,shapeName,shapeISO,shapeID,shapeGroup,shapeType,pop_count,geometry
0,13223003120321,Phu Quoc,None,VNM-ADM2-3_0_0-B967,VNM,ADM2,500.753008,"POLYGON ((103.46924 9.29731, 103.46924 9.31899..."
1,13223003120320,Phu Quoc,None,VNM-ADM2-3_0_0-B967,VNM,ADM2,6.106744,"POLYGON ((103.44727 9.29731, 103.44727 9.31899..."


In [ ]:
admin_grids_gdf.explore()

## Save to file

In [8]:
rollout_date = '-'.join(os.getcwd().split('/')[-2].split('-')[:3])
grid_save_path = f'./{rollout_date}-vn-rollout-grids.geojson'
admin_grids_gdf.to_file(grid_save_path, driver='GeoJSON', index=False)